<a href="https://colab.research.google.com/github/PavelNdjawa/Data_TDI/blob/master/CamairAI_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function, division

#!pip install efficientnet_lite0_pytorch_model

import os
import copy
from torch.utils.data import Dataset
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import time
import numpy as np
import pandas as pd
import os
import datetime as dt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics as metrics
#import efficientnet_lite0_pytorch_model


from fastai.vision import *
import fastai
from PIL import Image
import shutil
import imutils
import keras
# import fastai
# from fastai import vision

from copy import deepcopy


!pip install pretrainedmodels -q
import pretrainedmodels as pm
%matplotlib inline

     |████████████████████████████████| 61kB 4.8MB/s 


In [2]:
!nvidia-smi #Tesla T4

Tue Jun  1 19:32:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False.

True

In [4]:
import tensorflow as tf
tf.test.gpu_device_name()
# Standard output is '/device:GPU:0'

'/device:GPU:0'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# for reprocducibility
def seed_all(seed): 
  torch.manual_seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False


In [8]:
seed=0
seed_all(seed)

In [9]:
######### Read Train datalabels

path_train = '/content/drive/MyDrive/Competition/Train.csv'
path_test = '/content/drive/MyDrive/Competition/Test.csv'
 
train = pd.read_csv(path_train)
test = pd.read_csv(path_test)

In [10]:
train.head()

,img_IDs,target
0,ID_04UTZ83A24,0
1,ID_0URH2U2RON,1
2,ID_0ZEPOZGGQ1,0
3,ID_1595QTX5OA,1
4,ID_1RAE4SS5S6,1


In [11]:
seed_all(seed)
def create_label(images_path,df,train=True):

    for img in glob.glob(images_path+'/*.jpg'):
        im = img.split('/')[-1].split('.')[0]
        if im in Counter(df.img_IDs.to_list()):
            if train:
                label = df[df.img_IDs == im].target.values[0]
                if label==1:
                    shutil.copy2(images_path+im+'.jpg','Train/1')
                else:
                    shutil.copy2(images_path+im+'.jpg','Train/0')
            else:
                shutil.copy2(images_path+im+'.jpg','./Test')


In [12]:
%cd '/content/drive/MyDrive/Competition'

/content/drive/MyDrive/Competition


In [13]:
!pwd

/content/drive/MyDrive/Competition


In [14]:
#create_label('/content/drive/MyDrive/Competition/CAMAIRAI BOX READY/train/',1,train=True)

In [15]:
#create_label('/content/drive/MyDrive/Competition/CAMAIRAI BOX READY/train/',0,train=True)

In [16]:
#create_label('/content/drive/MyDrive/Competition/CAMAIRAI BOX READY/',test,train=False)
from copy import deepcopy

In [17]:
seed_all(seed)
def train_model(model, criterion, optimizer, scheduler, dataloaders,dataset_sizes,auc_bol,num_epochs=25):
    since = time.time()
    auc_list = []
    best_model_wts = deepcopy(model.state_dict())
    best_acc = 0.0
    best_auc = 0.0
    lambd = 0.1

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:

                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            auc = 0
            bs = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    outputs1 = F.softmax(outputs,dim=1)
                    max_prob, preds = torch.max(outputs1, dim=1)

                    prods, preds = torch.max(outputs, 1)
                    #print('prodd', max_prob.cpu().detach().numpy(),labels.data.cpu().detach().numpy())
                    
#                     if reg==True:
#                         regularization_loss = 0

#                         for param in model.parameters():
#                             regularization_loss+=torch.sum(abs(param))
                        
#                         loss = criterion(outputs, labels)+ lambd*regularization_loss
#                     else:
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                

                fpr, tpr, _= metrics.roc_curve(labels.data.cpu().detach().numpy(),                  
                                                 max_prob.cpu().detach().numpy(), pos_label=None)
                #print('vvv',Counter(labels.data.cpu().detach().numpy()))
                #print(labels.data.cpu().detach().numpy())
                #print(metrics.auc(fpr, tpr))
                auc += metrics.auc(fpr, tpr)
                bs+=1
                #print('auc', auc)

                # statistics
                running_loss += loss.item() * inputs.size(0)

                running_corrects += torch.sum(preds == labels.data)
                # auc_list.append(auc)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_auc = auc/bs

            print('{} Loss: {:.4f} Acc: {:.4f} Auc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc, epoch_auc))

            # deep copy the model
            if not auc_bol:
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = deepcopy(model.state_dict())
                    torch.save(best_model_wts , 'model.best')
            else:
                
                if phase== 'val' and epoch_auc> best_auc:
                    best_auc = epoch_auc
                    best_model_wts = model.state_dict()
                    torch.save(best_model_wts,'model.best')


        #print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    print()
    
    if not auc_bol:
        print('Best val Acc: {:4f}'.format(best_acc))
    else:
        print('Best val AUC : {:4f}'.format(best_auc))

    # load best model weights
    #model.load_state_dict(best_model_wts)
    model.load_state_dict(torch.load('model.best'))
    return model#, auc_list


In [18]:
# Data augmentation and normalization for training
# Just normalization for validation
seed_all(seed)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_transforms1 = {
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [19]:
seed_all(seed)
data_dir = '/content/drive/MyDrive/Competition/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                      for x in ['train']}

In [20]:
seed_all(seed)
image_datasets['train']

Dataset ImageFolder
    Number of datapoints: 100
    Root location: /content/drive/MyDrive/Competition/train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [21]:
# ### first_one
# seed_all(0)
# train_idx, valid_idx= train_test_split(
# np.arange(len(image_datasets['train'].targets)),
# test_size=0.2,
# shuffle=True,
# stratify=image_datasets['train'].targets,random_state=0)

# batch_size = 10
# train_loader = torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size,
#                                              sampler=ImbalancedDatasetSampler(image_datasets['train'],train_idx))
# valid_loader = torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size,
#                                              sampler=ImbalancedDatasetSampler(image_datasets['train'],valid_idx))


# dataloaders = {'train':train_loader, 'val': valid_loader}
# dataset_sizes = {x: len(dataloaders[x])*batch_size for x in ['train', 'val']}

In [22]:
### other_way
seed_all(seed)
class_names = image_datasets['train'].classes
dataset_sizes = {x: len(image_datasets[x]) for x in ['train']}


In [23]:
seed_all(seed)
train_idx, valid_idx = train_test_split(
np.arange(len(image_datasets['train'].targets)),
test_size=0.1,
shuffle=True,
stratify=image_datasets['train'].targets,random_state=0)


In [24]:
seed_all(seed)
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

sampler_={'train': train_sampler, 'val':valid_sampler}
batch_size = 10

train_loader = torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size,
                                             sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size,
                                             sampler=valid_sampler)

dataloaders = {'train':train_loader, 'val': valid_loader}
dataset_sizes = {x: len(dataloaders[x])*batch_size for x in ['train', 'val']}

In [25]:
seed_all(seed)
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


In [26]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [27]:
#############################
####### Train Models ########
#############################
seed_all(seed)
def build_models(pretrained=True, use_gpu=True, feature_extracting=True ):
  model = models.resnet50(pretrained=pretrained)
  set_parameter_requires_grad(model,feature_extracting)
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, 2)  
  if use_gpu:
      model = model.cuda()
  return model

feature_extract = False
model = build_models(True, True, feature_extract)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [28]:
seed_all(seed)
params_to_update = model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.0.conv3.weight
	 layer1.0.bn3.weight
	 layer1.0.bn3.bias
	 layer1.0.downsample.0.weight
	 layer1.0.downsample.1.weight
	 layer1.0.downsample.1.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer1.1.conv3.weight
	 layer1.1.bn3.weight
	 layer1.1.bn3.bias
	 layer1.2.conv1.weight
	 layer1.2.bn1.weight
	 layer1.2.bn1.bias
	 layer1.2.conv2.weight
	 layer1.2.bn2.weight
	 layer1.2.bn2.bias
	 layer1.2.conv3.weight
	 layer1.2.bn3.weight
	 layer1.2.bn3.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.conv3.weight
	 layer2.0.bn3.weight
	 layer2.0.bn3.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight

In [29]:
seed_all(seed)
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  seed_all(0)
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    #print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

In [30]:
seed_all(seed)
class FocalLoss(torch.nn.Module):
    '''Multi-class Focal loss implementation'''
    def __init__(self, gamma=2, weight=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.weight = weight

    def forward(self, input, target):
        """
        input: [N, C]
        target: [N, ]
        """
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        logpt = (1-pt)**self.gamma * logpt
        loss = F.nll_loss(logpt, target, self.weight)
        return loss  

In [31]:
seed_all(seed)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()#FocalLoss(gamma=2)#nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(params_to_update, lr=0.001, weight_decay=0,momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
###https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [32]:
seed_all(seed)
train_model(model, criterion, optimizer, exp_lr_scheduler,dataloaders,dataset_sizes,True, num_epochs=15)

Epoch 0/14
----------
train Loss: 0.7091 Acc: 0.5444 Auc: 0.5340
val Loss: 0.7120 Acc: 0.5000 Auc: 0.1667
Epoch 1/14
----------


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:800: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)


train Loss: 0.6213 Acc: 0.6667 Auc: nan
val Loss: 0.6862 Acc: 0.6000 Auc: 0.5417
Epoch 2/14
----------
train Loss: 0.5739 Acc: 0.6889 Auc: 0.8533
val Loss: 0.6707 Acc: 0.6000 Auc: 0.6250
Epoch 3/14
----------
train Loss: 0.5212 Acc: 0.7333 Auc: 0.4471
val Loss: 0.7859 Acc: 0.6000 Auc: 0.6667
Epoch 4/14
----------
train Loss: 0.5010 Acc: 0.7333 Auc: 0.8843
val Loss: 0.4418 Acc: 0.9000 Auc: 0.9167
Epoch 5/14
----------
train Loss: 0.4968 Acc: 0.7889 Auc: 0.4956
val Loss: 0.6760 Acc: 0.7000 Auc: 0.9583
Epoch 6/14
----------
train Loss: 0.5100 Acc: 0.7444 Auc: 0.8349
val Loss: 0.7495 Acc: 0.6000 Auc: 0.6667
Epoch 7/14
----------
train Loss: 0.3572 Acc: 0.8444 Auc: 0.6038
val Loss: 0.5405 Acc: 0.8000 Auc: 0.6667
Epoch 8/14
----------
train Loss: 0.3341 Acc: 0.8222 Auc: 0.6307
val Loss: 0.3812 Acc: 0.7000 Auc: 0.9583
Epoch 9/14
----------
train Loss: 0.3382 Acc: 0.8444 Auc: 0.7691
val Loss: 0.3639 Acc: 0.9000 Auc: 0.8750
Epoch 10/14
----------
train Loss: 0.3869 Acc: 0.8333 Auc: 0.6458
val L

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [33]:
#seed_all(seed)
#train_model(model,criterion,optimizer,exp_lr_scheduler,dataloaders,dataset_sizes,True,5)

In [34]:
seed_all(seed)
model.load_state_dict(torch.load('model.best'))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [35]:
### Don't need it

seed_all(seed)

def process_image(image_dir):
    # Process a PIL image for use in a PyTorch model
    # tensor.numpy().transpose(1, 2, 0)
    image = Image.open(image_dir)
    preprocess = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = preprocess(image)
    #print('image', image.size())
    # Convert 2D image to 1D vector
    image = np.expand_dims(image, 0)
    image = torch.from_numpy(image)
    inputs = image.to(device)


    return inputs  

In [36]:
seed_all(seed)
# Using our model to predict the label
def predict(image, model):
    # Pass the image through our model
    output = model(image)
    #print('image', image.size())
    #print('result',max(output))
    # Reverse the log function in our output

    output1 = F.softmax(output,dim=1)


    output = torch.exp(output)

    #print('output',torch.sum(output1).item())#.size())

    max_prob, preds = torch.max(output1, dim=1)

    #print('probs', max_prob)
    #print('preds', preds)
    #print(output.size())
    #print('222', output)
    # Get the top predicted class, and the output percentage for
    # that class
    probs, classes = output.topk(1, dim=1)
    #print('kk', classes)
    return max_prob.item(), preds.item()#probs.item(), classes.item()

In [37]:
seed_all(seed)
#don't need it
test_directory = '/content/drive/MyDrive/Competition/test'
predictions, probs ,test_image_fileName = [], [],[]
try:
    test_images = listdir(test_directory)
    for images in test_images:
        test_image_fileName.append(images)
        image = process_image(f'{test_directory}/{images}')
        top_prob, top_class = predict(image, model)
        probs.append(top_prob)
        predictions.append(class_names[top_class])
except Exception as e:
    print(e)

In [38]:
Counter(predictions)

Counter({'0': 1, '1': 33})

In [39]:
print(probs)

[0.8155629634857178, 0.9929948449134827, 0.9733119606971741, 0.995816171169281, 0.9531804919242859, 0.9940343499183655, 0.9238918423652649, 0.9834665656089783, 0.8758316040039062, 0.8961062431335449, 0.9884458780288696, 0.9967579245567322, 0.7822821140289307, 0.9022418260574341, 0.722368597984314, 0.9663171768188477, 0.9178600907325745, 0.9248434901237488, 0.971710741519928, 0.9950560331344604, 0.9505282044410706, 0.9860995411872864, 0.704914927482605, 0.9971331357955933, 0.9838416576385498, 0.991594135761261, 0.9984543323516846, 0.5179088115692139, 0.933698832988739, 0.7986231446266174, 0.7727144360542297, 0.9995158910751343, 0.9993231296539307, 0.9969424605369568]


In [40]:
#print(predictions)
len(test_image_fileName)

34

In [41]:
test_image_fileName_n = []
for elt in test_image_fileName:
  g = elt.split('.')
  test_image_fileName_n.append(g[0])

In [42]:
print("[INFO] Creating pandas dataframe")
submission_data = {"img_IDS":test_image_fileName_n,"target":probs,}
submission_data_frame = pd.DataFrame(submission_data)
submission_data_frame.head()

[INFO] Creating pandas dataframe


,img_IDS,target
0,ID_XDAH5643R4,0.815563
1,ID_8REUNTV3UL,0.992995
2,ID_AVGACN449M,0.973312
3,ID_37ULEGW12M,0.995816
4,ID_JXE4A2P3N9,0.953180


In [43]:
#import os
#submission_data_frame1=pd.DataFrame(
#    [os.path.splitext(f) for f in submission_data_frame.img_IDs])


In [44]:
print("[INFO] Saving Predicition to CSV")
submission_data_frame.to_csv('SampleSubmission_21.csv',columns=["img_IDS","target"], index = False)

[INFO] Saving Predicition to CSV
